In [1]:
import pandas as pd
import numpy as np
import json
from edgar import *
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle
import multiprocessing
set_identity('80fiokidrkiojiodjiofrjojioqq.com@gmail.com')

[22:37:50] INFO     Identity of the Edgar REST client set to                                            ]8;id=591510;file:///opt/anaconda3/envs/learn/lib/python3.9/site-packages/edgar/core.py\core.py]8;;\:]8;id=186193;file:///opt/anaconda3/envs/learn/lib/python3.9/site-packages/edgar/core.py#158\158]8;;\
                    [80fiokidrkiojiodjiofrjojioqq.com@gmail.com]                                                   

In [2]:
# Define lists of words
verbs_1 = ['freeze', 'froze', 'frozen', 'freezing']
nouns_1 = ['defined benefit', 'pension plan', 'retirement', 'postretirement', 'postemployment', 'pension', 'benefit']
years = [str(year) for year in range(1985, 2026)]
words_distance = 15

# Create a regex pattern
pattern_1 = r'\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b' + \
            r'|\b(?:' + '|'.join(re.escape(year) for year in years) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(verb) for verb in verbs_1) + r')\b(?:\W+\w+){0,' + str(words_distance) + r'}\W+\b(?:' + '|'.join(re.escape(noun) for noun in nouns_1) + r')\b'


def find_freeze(filing):
    
    try:
        result_1 = filing.search(pattern_1, regex=True).json() # search for keywords
    except:
        result_1 = None   

    # try:
    #     result_2 = filing.search(pattern_2, regex=True).json() # search for keywords
    # except:
    #     result_2 = None     

    return filing, (result_1, None)


In [3]:
for year in range(2020,2021):
    # open pcikle file to get the freeze collection
    with open(f'data/freeze_collection_{year}.pkl', 'rb') as f:
        freeze_collection = pickle.load(f)

    # loop through the freeze collection, key is the filing, only if result 1 is not None
    filing_list = [filing for filing, value in freeze_collection.items() if (value[0] is not None and value[0]['sections'])]

    res = Parallel(n_jobs=-1)(delayed(find_freeze)(filing) for filing in tqdm(filing_list))

    # save the result
    freeze_collection = dict(res)
        
    # save the collection
    with open(f'data/freeze_collection_{year}_alternative.pkl', 'wb') as f:
        pickle.dump(freeze_collection, f)

 15%|█▌        | 352/2273 [01:12<07:46,  4.11it/s]/opt/anaconda3/envs/learn/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 2273/2273 [13:08<00:00,  2.88it/s]
